In [19]:
from model import dp_syllabler
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
with open('data/x_val_ortho.pkl', 'rb') as file:
    x_val_ortho = pickle.load(file)
    
with open('data/x_val_ipa.pkl', 'rb') as file:
    x_val_ipa = pickle.load(file)

with open('data/y_val.pkl', 'rb') as file:
    y_val = pickle.load(file)
    
with open('data/x_tr_ortho.pkl', 'rb') as file:
    x_tr_ortho = pickle.load(file)

In [20]:
#ortho max encoder len = 45
#ipa max encoder len = 74

with open('data/e2i_ortho.pkl', 'rb') as file:
    e2i_ortho = pickle.load(file)
    
with open('data/e2i_ipa.pkl', 'rb') as file:
    e2i_ipa = pickle.load(file)

double_penetrator = dp_syllabler(e2i_ortho= e2i_ortho, e2i_ipa = e2i_ipa, ortho_input_size=45,ipa_input_size=74,latent_dim=32,embed_dim=32 ,max_feat=148)
double_penetrator.model.load_weights('data/32_false_double_pen_best_weights.h5')

In [21]:
def calc_f1(attempted, true, filename = 'data/evaluation_output_double_pen.txt'):
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    correct_num_char = 0
    total_checked = 0
    evaluation_file = open(filename, 'w+')
    for i in range(0, len(attempted)):
        total_checked += 1
        if (len(attempted[i]) == len(true[i])):
            correct_num_char += 1
            for j in range(0, len(attempted[i])):
                if(attempted[i][j] == true[i][j]):
                    if true[i][j] == 1:
                        true_neg += 1
                    elif true[i][j] == 2:
                        true_pos += 1
                else:
                    if true[i][j] == 1:
                        false_pos += 1
                    elif true[i][j] == 2:
                        false_neg += 1
    evaluation_file.write("Total checked: " + str(total_checked) + '\n')
    evaluation_file.write("Num correctly evaluated character count: " +  str(correct_num_char) + '\n')
    evaluation_file.write("True positives: " + str(true_pos) + '\n')
    evaluation_file.write("True negatives: " + str(true_neg) + '\n')
    evaluation_file.write("False positives: " + str(false_pos) + '\n')
    evaluation_file.write("False negatives: " + str(false_neg) + '\n')
    precision = true_pos/(true_pos+false_pos)
    recall = true_pos/(true_pos + false_neg)
    f_one = 2/((1/precision)+(1/recall))
    evaluation_file.write("precision: " + str(precision) + '\n')
    evaluation_file.write("recall: " + str(recall) +  '\n')
    evaluation_file.write("F1 scores: " + str(f_one) + '\n')
    evaluation_file.close()

In [22]:
reals = []
for x in y_val:
    reals += [[i for i in x if i !=0]]

In [23]:
attempts = []
for i in range(0, 5000):
    attempts += [double_penetrator.raw_syllabify(x_val_ortho[i], false_ipa[i])]
    print(i, end='\r')

In [15]:
attempts_stripped = []
for x in attempts:
    attempts_stripped += [[i for i in x if i !=0]]

In [25]:
calc_f1(attempts_stripped, reals, 'data/32_pure_false_evaluation_output.txt')
inconsistency_grab(attempts_stripped, reals, 'data/32_incorrect_syls.txt')

In [24]:
def inconsistency_grab(attempts, reals, filename = 'data/incorrect_syls.txt'):
    global converted_back_to_eng
    incorrect_counter = 0
    file = open(filename, 'w+')
    file.write('Attempt' + '\t' + 'Real' + '\n')
    for i in range(0, len(attempts)):
        if attempts[i] != reals[i]:
            incorrect_counter += 1
            a = double_penetrator.insert_syl(converted_back_to_eng[i], attempts[i])
            r = double_penetrator.insert_syl(converted_back_to_eng[i], reals[i])
            file.write(a + '\t' + r + '\n')
    file.write("Words with errors: %i"%incorrect_counter +'\n')
    file.write("Total evluated: %i"%len(attempts) +'\n')
    file.close()

In [ ]:
eng_conv_attempts = []
for i in range(0, len(attempts)):
    eng_conv_attempts += [double_penetrator.insert_syl(converted_back_to_eng[i], attempts[i])]

In [17]:
converted_back_to_eng = []
for x in x_val_ortho:
    real_word = ""
    for i in x:
            if i != 0:
                real_word += list(e2i_ortho.keys())[list(e2i_ortho.values()).index(i)]
    converted_back_to_eng += [real_word]

In [ ]:
# testing with falses
attempts_prime = []
for i in range(0, 5000):
    attempts_prime += [double_penetrator.raw_syllabify(x_val_ortho[i], np.array(false_ipa[i]))]
    print(i, end='\r')
    
attempts_prime_stripped = []
for x in attempts_prime:
    attempts_prime_stripped += [[i for i in x if i !=0]]
    
calc_f1(attempts_prime_stripped, reals, 'data/falses_evaluation_output_double_pen.txt')

In [ ]:
with open('data/eng_words.txt','w+') as file:
    for i in range(0,5000):
        file.write(converted_back_to_eng[i] + '\n')

In [ ]:
# falsing training data
tr_converted_back_to_eng = []
for x in x_tr_ortho:
    real_word = ""
    for i in x:
            if i != 0:
                real_word += list(e2i_ortho.keys())[list(e2i_ortho.values()).index(i)]
    tr_converted_back_to_eng += [real_word]


false_ipa_orig = []
false_ipa = []

for i in range(0,len(x_tr_ortho)):
    tried = []
    for q in tr_converted_back_to_eng[i]:
        tried += l2i[q]
    false_ipa_orig += [tried]
    
for x in false_ipa_orig:
    inted = []
    for y in x:
        inted += [e2i_ipa[y]]
    false_ipa += [inted]
        
    
false_ipa = pad_sequences(false_ipa, maxlen=74, padding='post')

with open('data/false_ipa.pkl', 'wb+') as file:
    pickle.dump(false_ipa, file)

In [9]:
l2i = {'a':'æ', 'b':'b', 'c':'k', 'd':'d', 'e':'e', 'f':'f' , 'g':'g', 'h':'h', 'i':'ɪ', 'j':'ʒ', 'k':'k', 'l':'l', 'm':'m', 'n':'n',
      'o':'ɑ', 'p':'p', 'q':'k', 'r':'ɹ', 's':'s', 't':'t', 'u':'ə', 'v':'v', 'w':'w', 'x':['k','s'], 'y':'j', 'z':'z'}

In [10]:
val_converted_back_to_eng = []
for x in x_val_ortho:
    real_word = ""
    for i in x:
            if i != 0:
                real_word += list(e2i_ortho.keys())[list(e2i_ortho.values()).index(i)]
    val_converted_back_to_eng += [real_word]

false_ipa_orig = []
false_ipa = []

for i in range(0,len(x_val_ortho)):
    tried = []
    for q in val_converted_back_to_eng[i]:
        tried += l2i[q]
    false_ipa_orig += [tried]
    
for x in false_ipa_orig:
    inted = []
    for y in x:
        inted += [e2i_ipa[y]]
    false_ipa += [inted]
        
    
false_ipa = pad_sequences(false_ipa, maxlen=74, padding='post')

with open('data/false_val_ipa.pkl', 'wb+') as file:
    pickle.dump(false_ipa, file)